<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/A(10%EA%B0%95_RNN(if_you_want_you).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import tensorflow as tf

# 1. 데이터 준비
text = "if you want you"  # 학습할 텍스트
chars = sorted(set(text))  # 고유한 문자들
char_to_idx = {char: idx for idx, char in enumerate(chars)}  # 문자 -> 인덱스 변환
idx_to_char = {idx: char for idx, char in enumerate(chars)}  # 인덱스 -> 문자 변환

# 데이터 전처리 (입력 시퀀스와 출력 시퀀스 생성)
input_seq = [char_to_idx[char] for char in text[:-1]]  # 마지막 문자는 제외
output_seq = [char_to_idx[char] for char in text[1:]]  # 첫 번째 문자는 제외

# 입력과 출력을 (1, 시퀀스 길이) 형태로 변환
input_seq = np.array(input_seq).reshape(1, -1)
output_seq = np.array(output_seq).reshape(1, -1)

vocab_size = len(chars)  # 고유한 문자 개수
embedding_dim = 8        # 임베딩 차원 수
input_length = input_seq.shape[1]  # 입력 시퀀스 길이

# 2. RNN 모델 정의
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=input_length),
    tf.keras.layers.SimpleRNN(64, return_sequences=True),
    tf.keras.layers.Dense(vocab_size, activation='softmax')
])

# 모델 컴파일
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# 3. 학습
model.fit(input_seq, output_seq, epochs=100)

# 4. 예측
def generate_text(model, start_string, num_generate):
    # 입력 텍스트를 숫자로 변환
    input_eval = [char_to_idx[char] for char in start_string]
    input_eval = np.array(input_eval).reshape(1, -1)

    generated_text = start_string

    # 문자를 예측
    for _ in range(num_generate):
        predictions = model(input_eval)
        predicted_id = np.argmax(predictions[0, -1, :])

        # 예측한 문자를 기존 텍스트에 추가
        predicted_char = idx_to_char[predicted_id]
        generated_text += predicted_char
        print("generated_text=", generated_text)

        # 다음 입력 시퀀스를 생성 (문자 추가)
        input_eval = np.append(input_eval, [[predicted_id]], axis=1)

    return generated_text

# 5. 예시 텍스트 생성
print(generate_text(model, start_string="if you ", num_generate=10))

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 2.3213
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 2.3032
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 2.2854
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 2.2673
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 2.2486
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 2.2285
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 2.2066
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 2.1822
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 2.1548
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 2.1238
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 2.0885
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 2.0485
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 2.0039
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1.9548
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 1.9020
Epoch 16/100
1/1 ━━━━━━━━━━━━━━